In [ ]:
import pandas as pd
import numpy as np

In [ ]:
game_df = pd.read_csv("raw_data/Twitch_game_data.csv")
global_df = pd.read_csv("raw_data/Twitch_global_data.csv")
streamers_df = pd.read_csv("raw_data/twitchdata-update.csv")

In [ ]:
def dateToDateTime(dataframe):
    df = dataframe.copy()
    df["day"] = 1
    df = df.set_index(pd.to_datetime(df[["year", "Month", "day"]]))
    df = df.drop(["year", "Month", "day"], axis=1)
    return df

## Global viewers chart

In [ ]:
Viewers = global_df[["year","Month","Avg_viewers"]]

In [ ]:
Viewers = dateToDateTime(Viewers)

In [ ]:
Viewers.to_csv("data/global_viewers.csv")

## Games views for ridgeplot

In [ ]:
non_games= ["Just Chatting","Talk Shows & Podcasts","Poker","Music","Creative","Pools, Hot Tubs, and Beaches","Science & Technology","Special Events","Art","Politics","Travel & Outdoors","Sports","ASMR","Food & Drink","Makers & Crafting","Beauty & Body Art","Software and Game Development"]

for ng in non_games:

    game_df.drop(game_df[game_df["Game"] == ng].index, inplace=True)


In [ ]:
game_avg = game_df.copy()
game_avg.drop(columns=["Rank","Hours_watched","Hours_Streamed","Peak_viewers","Peak_channels","Streamers","Avg_channels","Avg_viewer_ratio"] ,inplace=True)
game_avg["day"] = 1
game_avg["date"] = pd.to_datetime(game_avg[["Year", "Month", "day"]])
game_avg = game_avg.drop(["Year", "Month", "day"], axis=1)

In [ ]:
top_games = game_avg.groupby(['Game'])['Avg_viewers'].sum().sort_values(ascending=False)[:10].index.to_list()

In [ ]:
games = []

for game in top_games:
    games.append(game_avg[game_avg["Game"] == game])

In [ ]:
for g in games:
    g.rename(columns={"Avg_viewers" : g.iloc[0]["Game"]},inplace=True)
    g.drop(columns="Game", inplace=True)

In [ ]:
from functools import reduce

pd.concat(games)

top_games_avg_viewers = reduce(lambda  left,right: pd.merge(left,right, on="date", how="outer"), games).fillna(0)

In [ ]:
top_games_avg_viewers.to_csv("data/top_games_viewers.csv")

for i,game in enumerate(games):
    game.to_csv("data/"+ games[i]["Game"].iloc[0] +".csv")

## Average viewers of all games

In [ ]:
all_games = game_avg.groupby(['Game'])['Avg_viewers'].sum().sort_values(ascending=False).index.to_list()

games = []

for game in all_games:
    games.append(game_avg[game_avg["Game"] == game])

In [ ]:
for g in games:
    g.rename(columns={"Avg_viewers" : g.iloc[0]["Game"]},inplace=True)
    g.drop(columns="Game", inplace=True)


In [ ]:
all_games_avg_view = reduce(lambda  left,right: pd.merge(left,right, on="date", how="outer"), games).fillna(0)

In [ ]:
all_games_avg_view.to_csv("data/all_games_viewers.csv")

## Radar chart metrics

### total views

In [ ]:
radar_total = all_games_avg_view.sum(axis=0)

In [ ]:
radar_total.to_csv("data/radar_total_views.csv")

### mean views

In [ ]:
radar_avg = all_games_avg_view.sum(axis=0)

radar_avg.to_csv("data/radar_avg_views.csv")

### streamers

In [ ]:
game_stream = game_df.copy()
game_stream.drop(columns=["Rank","Hours_watched","Hours_Streamed","Peak_viewers","Peak_channels","Avg_viewers","Avg_channels","Avg_viewer_ratio"] ,inplace=True)

game_stream["day"] = 1
game_stream["date"] = pd.to_datetime(game_stream[["Year", "Month", "day"]])
game_stream = game_stream.drop(["Year", "Month", "day"], axis=1)

all_games = game_stream.groupby(['Game'])['Streamers'].sum().sort_values(ascending=False).index.to_list()

games = []

for game in all_games:
    games.append(game_stream[game_stream["Game"] == game])

for g in games:
    g.rename(columns={"Streamers" : g.iloc[0]["Game"]},inplace=True)
    g.drop(columns="Game", inplace=True)

all_games_streamers = reduce(lambda  left,right: pd.merge(left,right, on="date", how="outer"), games).fillna(0)

In [ ]:
mean_streamers = all_games_streamers.mean()

mean_streamers.to_csv("data/radar_total_streamers.csv")

### hours streamed / hours watched

In [ ]:
game_streamed = game_df.copy()
game_streamed.drop(columns=["Rank","Hours_watched","Streamers","Peak_viewers","Peak_channels","Avg_viewers","Avg_channels","Avg_viewer_ratio"] ,inplace=True)

game_streamed["day"] = 1
game_streamed["date"] = pd.to_datetime(game_streamed[["Year", "Month", "day"]])
game_streamed = game_streamed.drop(["Year", "Month", "day"], axis=1)

all_games = game_streamed.groupby(['Game'])['Hours_Streamed'].sum().sort_values(ascending=False).index.to_list()

games = []

for game in all_games:
    games.append(game_streamed[game_streamed["Game"] == game])

for g in games:
    g.rename(columns={"Hours_Streamed" : g.iloc[0]["Game"]},inplace=True)
    g.drop(columns="Game", inplace=True)

all_games_streamed = reduce(lambda  left,right: pd.merge(left,right, on="date", how="outer"), games).fillna(0)

In [73]:
all_games_streamedcopy = all_games_streamed.drop(columns="date").copy()

streamed = all_games_streamedcopy.applymap(lambda x : int(x.split()[0]) if not x == 0 else x).sum()



In [62]:
game_watched = game_df.copy()
game_watched.drop(columns=["Rank","Hours_Streamed","Streamers","Peak_viewers","Peak_channels","Avg_viewers","Avg_channels","Avg_viewer_ratio"] ,inplace=True)

game_watched["day"] = 1
game_watched["date"] = pd.to_datetime(game_watched[["Year", "Month", "day"]])
game_watched = game_watched.drop(["Year", "Month", "day"], axis=1)

all_games = game_watched.groupby(['Game'])['Hours_watched'].sum().sort_values(ascending=False).index.to_list()

games = []

for game in all_games:
    games.append(game_watched[game_watched["Game"] == game])

for g in games:
    g.rename(columns={"Hours_watched" : g.iloc[0]["Game"]},inplace=True)
    g.drop(columns="Game", inplace=True)

all_games_watched = reduce(lambda  left,right: pd.merge(left,right, on="date", how="outer"), games).fillna(0)

e:\Technologie\Programming\twitch-to-the-moon\.venv\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
e:\Technologie\Programming\twitch-to-the-moon\.venv\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [72]:
watched = all_games_watched.drop(columns="date").applymap(lambda x : int(x)).sum()

In [85]:
streamed.sort_index(ascending=False, inplace=True)
watched.sort_index(ascending=False, inplace=True)

ratio_watch = watched / streamed

In [86]:
ratio_watch.to_csv("data/radar_ratio_watch.csv")

theHunter: Call of the Wild     29.725549
some some convenience store     73.217168
skribbl.io                      31.476024
rFactor 2                      184.988249
osu!                             7.823594
                                  ...    
<U+014C>kami                     9.419343
7 Days to Die                   11.276369
60 Seconds!                    307.787250
60 Parsecs!                    283.710766
.hack//G.U. Last Recode          4.122469
Length: 1743, dtype: float64

## Domination of the Game

In [ ]:
global_avg_views = all_games_avg_view.drop(columns="date").sum(axis=1)

In [ ]:
all_games_avg_view["all_games"] = global_avg_views

In [ ]:
all_games_domination = pd.DataFrame()
for c in all_games_avg_view.columns:
    if c in ["date","all_games"]:
        all_games_domination[c] = all_games_avg_view[c]
    else:
        all_games_domination[c] = (all_games_avg_view[c] / all_games_avg_view["all_games"])*100

In [ ]:
all_games_domination.drop(columns="all_games", inplace=True)

In [ ]:
all_games_domination.to_csv("data/all_games_domination.csv")